### Visualizing_CO2_emissions


In this notebook, we aim to visualize and analyze CO₂ emissions per capita, total CO₂ emissions, population, and GDP per capita for European countries from 1970 to 2022. Through animated and interactive visualizations, We will explore how emissions have changed over time and how they relate to population and economic factors across Europe.

We use the datasets that we have calculated in the notebooks GDP_CO2_emissions.py, CO2_emissions_by_population.py and CO2_emissions_by_country. We have all the datasets that we need there to visualize  CO₂ emissions per capita, total CO₂ emissions and GDP per capita over the years.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
import matplotlib as mpl
import seaborn as sns
from ipywidgets import interact, IntSlider
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

In [2]:



data_root = './data/'
data_root_world = './data/cultural/'

df_co2 = pd.read_csv(f'{data_root}co2_emmisions_complicated.csv')
df_pop = pd.read_csv(f'{data_root}world_population.csv')
df_gdp = pd.read_csv(f'{data_root}co2-emissions-vs-gdp.csv')


# Filtering the data only for European countries
df_co2_europe = df_co2[df_co2['Region'].str.contains('Europe', case=False, na=False)]

# Add Russia
df_russia = df_co2[df_co2['Name'].str.contains('Russian Federation', case=False, na=False)].copy()
df_russia['Name'] = df_russia['Name'].replace('Russian Federation', 'Russia')

# Add Ukraine
df_ukraine = df_co2[df_co2['Name'].str.contains('Ukraine', case=False, na=False)].copy()

# Add Belarus
df_belarus = df_co2[df_co2['Name'].str.contains('Belarus', case=False, na=False)].copy()

# Add Moldova
df_moldova = df_co2[df_co2['Name'].str.contains('Moldova', case=False, na=False)].copy()
df_moldova['Name'] = df_moldova['Name'].replace('Moldova, Republic of', 'Moldova')

# Combine all
df_co2_europe = pd.concat([df_co2_europe, df_russia, df_ukraine, df_belarus, df_moldova]).drop_duplicates()

df_pop_europe = df_pop[df_pop['Continent'] == 'Europe']



row = df_co2_europe[df_co2_europe['Name'] == 'Serbia and Montenegro'].copy()

serbia_row = row.copy()
montenegro_row = row.copy()

serbia_row['Country_code'] = 'SRB'
serbia_row['Name'] = 'Serbia'

montenegro_row['Country_code'] = 'MNE'
montenegro_row['Name'] = 'Montenegro'

year_columns = [col for col in df_co2_europe.columns if col.isdigit()]

# Split the CO2 data by 85% and 15% because Serbia is much bigger than Montenegro by area and by population
serbia_row[year_columns] = row[year_columns] * 0.96
montenegro_row[year_columns] = row[year_columns] * 0.04

# Drop the original Serbia and Montenegro row
df_co2_europe = df_co2_europe[df_co2_europe['Name'] != 'Serbia and Montenegro']

# Append the two new rows
df_co2_europe = pd.concat([df_co2_europe, serbia_row, montenegro_row], ignore_index=True)




years = ['1970', '1980', '1990', '2000', '2010', '2015', '2020', '2022']

df_co2_filtered = df_co2_europe[['Name'] + years]

pop_years = [f"{year} Population" for year in [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]]
df_pop_filtered = df_pop_europe[['Country/Territory'] + pop_years]

# total CO2 per year
co2_total = df_co2_filtered[years].sum()

# total population per year
pop_total = df_pop_filtered[pop_years].sum()
pop_total.index = years # index -> years




df_co2_filtered.head(3)






,Name,1970,1980,1990,2000,2010,2015,2020,2022
0,Albania,4844.988068,8073.499079,6644.969301,3232.531279,4556.850488,4863.525143,4560.776421,4609.009805
1,Austria,52967.212730,61452.922330,61559.285850,66690.721000,73795.780480,67369.130490,62599.758260,61223.713120
2,Belgium,139256.731700,137777.899900,115808.577800,124969.640800,114977.049400,102379.091800,92267.020120,89739.264640


In [3]:
df_pop_filtered.head(3)

,Country/Territory,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,2015 Population,2020 Population,2022 Population
1,Albania,2324731,2941651,3295066,3182021,2913399,2882481,2866849,2842321
4,Andorra,19860,35611,53569,66097,71519,71746,77700,79824
12,Austria,7465301,7547561,7678729,8010428,8362829,8642421,8907777,8939617


In [4]:
co2__ = df_co2_europe[['Name']]
pop__ = df_pop_europe[['Country/Territory']]

df_merged = pd.merge(df_co2_filtered,df_pop_filtered, left_on='Name', right_on='Country/Territory', how='inner')

df_merged

# Calculate CO2 per capita for each year
df_per_capita = df_merged.copy()

for year in years:
    co2_col = year
    pop_col = f"{year} Population"
    df_per_capita[year] = df_per_capita[co2_col] / df_per_capita[pop_col]

# Keep only country names and per capita columns
df_per_capita = df_per_capita[['Name'] + years]
df_per_capita.head(3)



,Name,1970,1980,1990,2000,2010,2015,2020,2022
0,Albania,0.002084,0.002745,0.002017,0.001016,0.001564,0.001687,0.001591,0.001622
1,Austria,0.007095,0.008142,0.008017,0.008325,0.008824,0.007795,0.007028,0.006849
2,Belgium,0.014462,0.014018,0.011628,0.012175,0.010570,0.009102,0.007980,0.007699


In [5]:
import geodatasets
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from ipywidgets import interact, IntSlider
import matplotlib.gridspec as gridspec
# Your CO2 data
df = df_per_capita

name_corrections = {
    "Czech Republic": "Czechia",
    "Bosnia and Herzovina": "Bosnia and Herz.",
    "Macedonia": "Macedonia",
    "United Kingdom": "United Kingdom",
    "Russia": "Russia",
    "Moldova": "Moldova",
    "Slovakia": "Slovakia",
    "Serbia": "Republic of Serbia",
    "Montenegro": "Montenegro",
    "Germany": "Germany"
}

df_melted = df.melt(id_vars=['Name'], var_name='Year', value_name='CO2_per_capita')
df_melted["Name"] = df_melted["Name"].replace(name_corrections)

# Convert 'Year' column to int
df_melted['Year'] = df_melted['Year'].astype(int)

# Your exact years list
years = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]

world = gpd.read_file(f'{data_root_world}ne_110m_admin_0_countries.shp')
europe = world[world['CONTINENT'] == 'Europe']

vmin = df_melted['CO2_per_capita'].min()
vmax = df_melted['CO2_per_capita'].max()
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = plt.cm.Reds

bounds = europe.total_bounds  # [minx, miny, maxx, maxy]

def plot_year(year):
    fig = plt.figure(figsize=(20, 10))  # bigger figure
    gs = gridspec.GridSpec(1, 2, width_ratios=[30, 1], wspace=0.05)  # 20:1 ratio for map and colorbar
    fig.subplots_adjust(left=0.02, right= 0.95, top=0.95, bottom=0.05)

    ax = fig.add_subplot(gs[0])  # big axis for map
    cax = fig.add_subplot(gs[1])  # narrow axis for colorbar

    df_year = df_melted[df_melted['Year'] == year]
    merged = europe.merge(df_year, how='left', left_on='ADMIN', right_on='Name')

    ax.clear()
    merged.plot(column='CO2_per_capita', cmap=cmap, linewidth=0.8, ax=ax, edgecolor='0.8',
                missing_kwds={'color': 'lightgrey'}, norm=norm)

    bounds = europe.total_bounds
    ax.set_xlim(-10, 170)
    ax.set_ylim(20, 90)

    ax.set_title(f"CO₂ Emissions per Capita in Europe ({year})", fontsize=22)
    ax.axis('off')

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax, orientation="vertical")
    cbar.set_label("CO₂ per Capita (tons)", fontsize=16)

    plt.show()

interact(plot_year, year=IntSlider(min=years[0], max=years[-1], step=10, value=years[0]));


C:\Users\HP\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


interactive(children=(IntSlider(value=1970, description='year', max=2022, min=1970, step=10), Output()), _dom_…

In [6]:
df_emissions_anim1 = df_co2_filtered.copy()

# Correct country names
name_corrections_anim1 = {
    "Czech Republic": "Czechia",
    "Macedonia": "Macedonia",
    "United Kingdom": "United Kingdom",
    "Russia": "Russia",
    "Moldova": "Moldova",
    "Slovakia": "Slovakia",
    "Serbia": "Republic of Serbia",
    "Montenegro": "Montenegro",
    "Germany": "Germany"
}

df_emissions_anim1['Name'] = df_emissions_anim1['Name'].replace(name_corrections_anim1)

# Melt data into long format
df_emissions_long1 = df_emissions_anim1.melt(id_vars=['Name'], var_name='Year', value_name='Total_CO2')
df_emissions_long1['Year'] = df_emissions_long1['Year'].astype(int)

# Define years
years_anim1 = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]

# Normalize colors
vmin_anim1 = df_emissions_long1['Total_CO2'].min()
vmax_anim1 = df_emissions_long1['Total_CO2'].max()
norm_anim1 = mcolors.Normalize(vmin=vmin_anim1, vmax=vmax_anim1)
cmap_anim1 = plt.cm.Reds

def plot_emissions_anim1(year):
    fig = plt.figure(figsize=(20, 10))
    gs = gridspec.GridSpec(1, 2, width_ratios=[30, 1], wspace=0.05)
    fig.subplots_adjust(left=0.02, right=0.95, top=0.95, bottom=0.05)

    ax = fig.add_subplot(gs[0])
    cax = fig.add_subplot(gs[1])

    df_year_anim1 = df_emissions_long1[df_emissions_long1['Year'] == year]
    merged_anim1 = europe.merge(df_year_anim1, how='left', left_on='ADMIN', right_on='Name')

    ax.clear()
    merged_anim1.plot(column='Total_CO2', cmap=cmap_anim1, linewidth=0.8, ax=ax, edgecolor='0.8',
                      missing_kwds={'color': 'lightgrey'}, norm=norm_anim1)

    ax.set_xlim(-10, 170)
    ax.set_ylim(20, 90)
    ax.set_title(f"Total CO₂ Emissions in Europe ({year})", fontsize=22)
    ax.axis('off')

    sm = plt.cm.ScalarMappable(cmap=cmap_anim1, norm=norm_anim1)
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax, orientation="vertical")
    cbar.set_label("Total CO₂ Emissions (tons)", fontsize=16)

    plt.show()

interact(plot_emissions_anim1, year=IntSlider(min=years_anim1[0], max=years_anim1[-1], step=10, value=years_anim1[0]))

# Slider widget
year_slider_anim1 = widgets.SelectionSlider(
    options=years_anim1,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def on_year_change_anim1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plot_emissions_anim1(change['new'])

year_slider_anim1.observe(on_year_change_anim1)

# Country sets (for diagnostics, if needed)
map_countries_anim1 = set(europe['ADMIN'].unique())
data_countries_anim1 = set(df_emissions_long1['Name'].unique())


interactive(children=(IntSlider(value=1970, description='year', max=2022, min=1970, step=10), Output()), _dom_…

In [7]:
european_countries = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium",
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary",
    "Iceland", "Ireland", "Italy", "Kazakhstan", "Kosovo", "Latvia", "Liechtenstein",
    "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro",
    "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", "Romania",
    "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden",
    "Switzerland", "Turkey", "Ukraine", "United Kingdom", "Vatican"
]

# Filter for European countries
df_gdp_europe = df_gdp[df_gdp['Entity'].isin(european_countries)].copy()


df_gdp_europe.columns = df_gdp_europe.columns.str.strip()
df_gdp_europe = df_gdp_europe.dropna(subset=['GDP per capita'])
# Convert GDP per capita to numeric if needed
df_gdp_europe['GDP per capita'] = pd.to_numeric(df_gdp_europe['GDP per capita'], errors='coerce')
df_filtered = df_gdp_europe[(df_gdp_europe['Year'] >= 1970) & (df_gdp_europe['Year'] <= 2023)].copy()

# columns to numbers, avoid errors
df_filtered['Annual CO₂ emissions (per capita)'] = pd.to_numeric(df_filtered['Annual CO₂ emissions (per capita)'], errors='coerce')
df_filtered['GDP per capita'] = pd.to_numeric(df_filtered['GDP per capita'], errors='coerce')

# Drop rows with missing values
df_filtered = df_filtered.dropna(subset=['Annual CO₂ emissions (per capita)', 'GDP per capita'])

# Group by year and sum values
annual_co2_sum = df_filtered.groupby('Year')['Annual CO₂ emissions (per capita)'].sum()
annual_gdp_sum = df_filtered.groupby('Year')['GDP per capita'].sum()

years = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]



df_years = df_filtered[df_filtered['Year'].isin(years)]

df_pivot = df_years.pivot(index='Entity', columns='Year', values='GDP per capita')

df_pivot = df_pivot.sort_index()
df_pivot = df_pivot[sorted(df_pivot.columns)]
df_pivot.head(3)



 



Year,1970,1980,1990,2000,2010,2015,2020,2022
Entity,,,,,,,,
Albania,3194.0,3741.0,3983.0,4808.4795,9222.973,10192.529,11372.541,12978.101
Armenia,NaN,9291.0,9669.0,5139.8257,8330.812,10076.352,11546.176,13837.577
Austria,15537.0,21932.0,26930.0,34796.2580,40288.348,41255.715,40503.040,43792.855


In [8]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from ipywidgets import interact, IntSlider, widgets

# Copy df_filtered to avoid changing original
df_gdp_anim2 = df_filtered.copy()

# Fix country names if needed
name_corrections_gdp2 = {
    "Czech Republic": "Czechia",
    "Macedonia": "Macedonia",
    "United Kingdom": "United Kingdom",
    "Russia": "Russia",
    "Moldova": "Moldova",
    "Slovakia": "Slovakia",
    "Serbia": "Republic of Serbia",
    "Montenegro": "Montenegro",
    "Germany": "Germany"
}

df_gdp_anim2['Entity'] = df_gdp_anim2['Entity'].replace(name_corrections_gdp2)

# Filter only required years
years_gdp2 = [1970, 1980, 1990, 2000, 2010, 2015, 2020, 2022]
df_years_gdp2 = df_gdp_anim2[df_gdp_anim2['Year'].isin(years_gdp2)]

# Normalize colors for GDP per capita
vmin_gdp2 = df_years_gdp2['GDP per capita'].min()
vmax_gdp2 = df_years_gdp2['GDP per capita'].max()
norm_gdp2 = mcolors.Normalize(vmin=vmin_gdp2, vmax=vmax_gdp2)
cmap_gdp2 = plt.cm.viridis

def plot_gdp_per_capita_anim2(year):
    fig = plt.figure(figsize=(20, 10))
    gs = gridspec.GridSpec(1, 2, width_ratios=[30, 1], wspace=0.05)
    fig.subplots_adjust(left=0.02, right=0.95, top=0.95, bottom=0.05)

    ax = fig.add_subplot(gs[0])
    cax = fig.add_subplot(gs[1])

    df_year_gdp2 = df_years_gdp2[df_years_gdp2['Year'] == year]
    merged_gdp2 = europe.merge(df_year_gdp2, how='left', left_on='ADMIN', right_on='Entity')

    ax.clear()
    merged_gdp2.plot(column='GDP per capita', cmap=cmap_gdp2, linewidth=0.8, ax=ax, edgecolor='0.8',
                    missing_kwds={'color': 'lightgrey'}, norm=norm_gdp2)

    ax.set_xlim(-10, 170)
    ax.set_ylim(20, 90)

    ax.set_title(f"GDP per Capita in Europe ({year})", fontsize=22)
    ax.axis('off')

    sm = plt.cm.ScalarMappable(cmap=cmap_gdp2, norm=norm_gdp2)
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax, orientation="vertical")
    cbar.set_label("GDP per Capita (USD)", fontsize=16)

    plt.show()

# Interactive slider
interact(plot_gdp_per_capita_anim2, year=widgets.SelectionSlider(
    options=years_gdp2,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
));


interactive(children=(SelectionSlider(continuous_update=False, description='Year:', options=(1970, 1980, 1990,…